In [ ]:
import numpy as np
from scipy.signal import butter, lfilter, welch
from muselsl import list_muses, stream, view
import pyautogui
from pylsl import StreamInlet, resolve_byprop


In [ ]:
import pyautogui
from time import sleep

# Give yourself a couple of seconds to switch to the game window
print("Switch to the game window. The spacebar will be pressed automatically after 5 seconds...")
sleep(5)

# Press the spacebar - this should make the character in the game jump
pyautogui.press('space')
print("The spacebar was pressed.")

# Run this block to see if the Dino jumps. Make sure the Dino game window is active and in focus.


Switch to the game window. The spacebar will be pressed automatically after 5 seconds...
The spacebar was pressed.


# Parameters for Tweaking:

1.  FS (Sampling Frequency):

    Represents how many EEG data points are collected in one second.
    It must match your EEG device's data output rate.
    Typical values are 256 Hz, 512 Hz, etc.

2. BETA_LOW, BETA_HIGH (Beta Band Range):

    The frequency range considered to represent the beta waves, usually associated with active, busy, or anxious thinking and active concentration.
    Typical beta band range is between 13 and 30 Hz.

3. BUFFER_LENGTH:

    The length of the EEG data buffer, in samples (FS * seconds).
    Determines how many seconds of EEG data you're analyzing at once.
    A longer buffer includes more data for analysis, which can smooth out noise but may delay responsiveness.

4. HIGH_CONCENTRATION_THRESHOLD:

    The threshold value against which the beta band power is compared to decide if it represents a high concentration state.
    Must be calibrated against baseline measurements of beta power when the user is concentrated versus when they are not.

5.  DEBOUNCE_TIME:

    A mechanism to prevent the system from triggering another jump too quickly after the previous one.
    It is the minimum amount of time that must elapse before the program can trigger another action.

6. Order of the Butterworth Bandpass Filter:

    Higher order filters have steeper roll-offs at the expense of increased processing time and potential for introducing phase distortions.
    A typical order might be 4 or 5, but this can be increased if a sharper frequency cutoff is needed.

In [2]:
# Constants
FS = 256  # Hz, sampling frequency
BETA_LOW, BETA_HIGH = 13, 30  # Beta band range in Hz
BUFFER_LENGTH = 5 * FS  # 5 seconds of data
HIGH_CONCENTRATION_THRESHOLD = 0.01  # Example threshold, calibrate this

# Design the Butterworth bandpass filter
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

# Apply the bandpass filter
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data, axis=0)
    return y

# Calculate the beta band power
def beta_band_power(data, fs):
    freqs, psd = welch(data, fs=fs, axis=0)
    idx_beta = np.logical_and(freqs >= BETA_LOW, freqs <= BETA_HIGH)
    beta_power = np.mean(psd[idx_beta], axis=0)
    return beta_power

# Main processing function
def process_eeg_data(inlet):
    # Initialize buffer
    eeg_buffer = np.zeros((BUFFER_LENGTH, inlet.info().channel_count()))
    
    while True:
        # Pull sample from inlet
        sample, timestamp = inlet.pull_sample(timeout=1.0)
        if sample:
            # Update the buffer
            eeg_buffer = np.roll(eeg_buffer, -1, axis=0)
            eeg_buffer[-1, :] = sample

            # Check if buffer is full
            if np.all(eeg_buffer[:, 0] != 0):
                # Preprocess the buffer - apply bandpass filter
                filtered_data = bandpass_filter(eeg_buffer, BETA_LOW, BETA_HIGH, FS)

                # Calculate beta band power
                beta_power = beta_band_power(filtered_data, FS)

                # Determine high concentration
                high_concentration = beta_power > HIGH_CONCENTRATION_THRESHOLD

                # Output high concentration status
                print(f"High concentration detected: {high_concentration}")
                # Here you would trigger a game action if high concentration is detected
                # trigger_game_action('move') if high_concentration else None
                # Give yourself a couple of seconds to switch to the game window
                print("Switch to the game window. The spacebar will be pressed automatically ...")
                #sleep(5)
                pyautogui.press('space')

# Resolve the stream
print("Looking for an EEG stream...")
streams = resolve_byprop('type', 'EEG', timeout=2)

if streams:
    inlet = StreamInlet(streams[0])
    print("Stream found and connected")
    process_eeg_data(inlet)  # Start processing EEG data
else:
    print("No streams found")

Looking for an EEG stream...
Stream found and connected
High concentration detected: [ True  True  True  True  True]
Switch to the game window. The spacebar will be pressed automatically ...
High concentration detected: [ True  True  True  True  True]
Switch to the game window. The spacebar will be pressed automatically ...
High concentration detected: [ True  True  True  True  True]
Switch to the game window. The spacebar will be pressed automatically ...
High concentration detected: [ True  True  True  True  True]
Switch to the game window. The spacebar will be pressed automatically ...
High concentration detected: [ True  True  True  True  True]
Switch to the game window. The spacebar will be pressed automatically ...
High concentration detected: [ True  True  True  True  True]
Switch to the game window. The spacebar will be pressed automatically ...
High concentration detected: [ True  True  True  True  True]
Switch to the game window. The spacebar will be pressed automatically ...

KeyboardInterrupt: 